---
title       : "Extremum estimation"
subtitle    :
author      : Paul Schrimpf
date        : `j using Dates; print(Dates.today())`
bibliography: "ee.bib"
---

<a rel="license"
href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative
Commons License" style="border-width:0"
src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png"
/></a><br />This work is licensed under a <a rel="license"
href="http://creativecommons.org/licenses/by-sa/4.0/">Creative
Commons Attribution-ShareAlike 4.0 International License</a>.

### About this document {-}

This document was created using Weave.jl. The code is available in
[the course github
repository](https://github.com/ubcecon/ECON628_2018). The same
document generates both static webpages and associated jupyter
notebooks.

$$
\def\indep{\perp\!\!\!\perp}
\def\Er{\mathrm{E}}
\def\R{\mathbb{R}}
\def\En{{\mathbb{E}_n}}
\def\Pr{\mathrm{P}}
\newcommand{\norm}[1]{\left\Vert {#1} \right\Vert}
\newcommand{\abs}[1]{\left\vert {#1} \right\vert}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\def\inprob{\,{\buildrel p \over \rightarrow}\,} 
\def\indist{\,{\buildrel d \over \rightarrow}\,} 
$$

# Introduction

As you saw in 627, many, perhaps most, estimators in econometrics are
extrumem estimators. That is, many estimators are defined by
$$
\hat{\theta} = \argmax_{\theta \in \Theta} \hat{Q}_n(\theta)
$$
where $\hat{Q}_n(\theta)$ is some objective function that depends on
data. Examples include maximum likelihood,
$$
\hat{Q}_n(\theta) = \frac{1}{n} \sum_{i=1}^n f(z_i | \theta)
$$
GMM,
$$
\hat{Q}_n(\theta) = \left(\frac{1}{n} \sum_{i=1}^n g(z_i,
\theta)\right)' \hat{W} \left(\frac{1}{n} \sum_{i=1}^n g(z_i,
\theta)\right)
$$
and nonlinear least squares
$$
\hat{Q}_n(\theta) = \frac{1}{n} \sum_{i=1}^n (y_i - h(x_i,\theta))^2.
$$
See @newey1994 for more details and examples.

## Example: logit

As a simple example, let's look look at some code for estimating a
logit.

In [ ]:
using Distributions, Optim, ForwardDiff, BenchmarkTools
function simulate_logit(observations, β)
  x = randn(observations, length(β))
  y = (x*β + rand(Logistic(), observations)) .>= 0.0
  return((y=y,x=x))
end 

function logit_likelihood(β,y,x)
  p = map(xb -> cdf(Logistic(),xb), x*β)
  sum(log.(ifelse.(y, p, 1.0 .- p)))
end

n = 500
k = 3
β0 = ones(k)
(y,x) = simulate_logit(n,β0)
Q = β -> -logit_likelihood(β,y,x)
Q(β0)

In [ ]:
@btime optimize(Q, zeros(k), NelderMead())
@btime optimize(Q, zeros(k), BFGS(), autodiff = :forward)
@btime optimize(Q, zeros(k), NewtonTrustRegion(), autodiff =:forward)

### Aside: Reverse mode automatic differentiation

For functions $f:\R^n \to \R^m$, the work for forward
automatic differentiation increases linearly with $n$. This is because
forward automatic differentiation applies the chain rule to each of the $n$
inputs. An alternative, is reverse automatic differentiation. Reverse
automatic differentiation is also based on the chain rule, but it
works backward from $f$ through intermediate steps back to $x$. The
work needed here scales linearly with $m$. Since optimization problems
have $m=1$, reverse automatic differentiation can often work well. The
downsides of reverse automatic differentiation are that: (1) it can
require a large amount of memory and (2) it is more difficult to
implement. There are handful of Julia packages that provide reverse
automatic differentiation, but they all have some limitations in terms
of what functions thay can differentiate. Flux.jl and JuMP.jl, which were included
as examples in the Solvers, Optimizers, and Automatic Differentiation
notebook, uses reverse automatic differentiation. ReverseDiff.jl is
another package for reverse automatic differentiation. ReverseDiff.jl
places fewer restrictions on the functions that it can differentiate,
but its creators no longer plan to update it.

In [ ]:
using ReverseDiff, Optim, BenchmarkTools
dQr = β->ReverseDiff.gradient(Q,β)
dQf = β->ForwardDiff.gradient(Q,β)

hcat(dQr(β0) , dQf(β0))

@btime optimize(Q, dQf, zeros(k), BFGS(); inplace=false)
@btime optimize(Q, dQr, zeros(k), BFGS(); inplace=false)

k = 100
β0 = ones(k)
(y,x) = simulate_logit(n,β0)
Q = β -> -logit_likelihood(β,y,x)
dQr = β->ReverseDiff.gradient(Q,β)
dQf = β->ForwardDiff.gradient(Q,β)
hcat(dQr(β0) , dQf(β0))
@btime optimize(Q, dQf, zeros(k), BFGS(); inplace=false)
@btime optimize(Q, dQr, zeros(k), BFGS(); inplace=false)

# Review of extremum estimator theory

This is based on @newey1994. You should already be familiar with this
from 627, so we will just state some basic "high-level" conditions for
consistency and asymptotic normality. 

## Consistency

<div class="theorem">
**Consistency for extremum estimators**: assume

1. $\hat{Q}_n(\theta)$ converges uniformly in probability to
   $Q_0(\theta)$
2. $Q_0(\theta)$ is uniquely maximized at $\theta_0$.
3. $\Theta$ is compact and $Q_0(\theta)$ is continuous.

Then $\hat{\theta} \inprob \theta_0$
</div>


## Asymptotic normality

<div class="theorem">
**Asymptotic normality for extremum estimators**: assume
1. $\hat{\theta} \inprob \theta_0$
2. $\theta_0 \in interior(\Theta)$
3. $\hat{Q}_n(\theta)$ is twice continuously differentiable in open $N$
   containing $\theta$, and $\sup_{\theta \in N} \Vert \nabla^2
                             \hat{Q}_n(\theta) - H(\theta) \Vert
                             \inprob 0$ with $H(\theta_0)$ nonsingular
4. $\sqrt{n} \nabla \hat{Q}_n(\theta_0) \indist N(0,\Sigma)$
Then $\sqrt{n} (\hat{\theta} - \theta_0) \indist N\left(0,H^{-1} \Sigma
  H^{-1} \right)$
</div>

Implementing this in Julia using automatic differentiation is pretty
straightforward.

In [ ]:
function logit_likei(β,y,x)
  p = map(xb -> cdf(Logistic(),xb), x*β)
  log.(ifelse.(y, p, 1.0 .- p))
end

function logit_likelihood(β,y,x)
  mean(logit_likei(β,y,x))
end

n = 1000
k = 3
β0 = ones(k)
(y,x) = simulate_logit(n,β0)
      
Q = β -> -logit_likelihood(β,y,x)      
optres = optimize(Q, zeros(k), NewtonTrustRegion(), autodiff =:forward)
βhat = optres.minimizer

function asymptotic_variance(Q,dQi, θ)
  gi = dQi(θ)
  Σ = gi'*gi/size(gi)[1]
  H = ForwardDiff.hessian(Q,θ)
  invH = inv(H)
  (variance=invH*Σ*invH, Σ=Σ, invH=invH)
end

avar=asymptotic_variance(θ->logit_likelihood(θ,y,x),
                         θ->ForwardDiff.jacobian(β->logit_likei(β,y,x),θ),βhat)
@show avar.variance/n
@show -avar.invH/n
@show inv(avar.Σ)/n

For maximum likelihood, the information equality says $-H = \Sigma$,
so the three expressions above have the same probability limit, and
are each consistent estimates of the variance of $\hat{\theta}$.

The code above is for demonstration and learning. If we really wanted
to estimate a logit for research, it would be better to use a
well-tested package. Here's how to estimate  a logit using GLM.jl.

In [ ]:
using GLM, DataFrames
df = DataFrame(x)
df[:y] = y
glmest=glm(@formula(y ~ -1 + x1+x2+x3), df, Binomial(),LogitLink())
@show glmest
@show vcov(glmest)

## Delta method

In many models, we are interested in some transformation of the
parameters in addition to the parameters themselves. For example, in a
logit, we might want to report marginal effects in addition to the
coefficients. In structural models, we typically use the parameter
estimates to conduct counterfactual simulations. In many 
situations we are more interested these transformation(s) of
parameters than in the parameters themselves. The delta method is one
convenient way to approximate the distribution of transformations of 
the model parameters. 

<div class="theorem">
**Delta method** assume:
1. $\sqrt{n} (\hat{\theta} - \theta_0) \indist N(0,\Omega)$
2. $g: \R^k \to \R^m$ is continuously differentiable
Then $\sqrt{n}(g(\hat{\theta}) - g(\theta_0)) \indist N(0, \nabla
      g(\theta_0)^T \Omega \nabla g(\theta_0)$
</div>

The following code uses the delta method to plot a 90% pointwise
confidence band around the estimate marginal effect of one of the
regressors.

In [ ]:
using LinearAlgebra
function logit_mfx(β,x)
  ForwardDiff.jacobian(β-> map(xb -> cdf(Logistic(),xb), x*β), β)  
end

function delta_method(g, θ, Ω)
  dG = ForwardDiff.jacobian(θ->g(θ),θ)
  dG*Ω*dG'  
end

nfx = 100
xmfx = zeros(nfx,3)
xmfx[:,1] .= -3.0:(6.0/(nfx-1)):3.0

mfx = logit_mfx(βhat,xmfx)
vmfx = delta_method(β->logit_mfx(β,xmfx)[:,1], βhat, avar.variance/n)
sdfx = sqrt.(diag(vmfx))

using Plots
Plots.gr()
plot(xmfx[:,1],mfx[:,1],ribbon=quantile(Normal(),0.95)*sdfx,fillalpha=0.5,
     xlabel="x[1]", ylabel="dP(y=1|x)/dx[1]", legend=false,
     title="Marginal effect of x[1] when x[2:k]=0")

The same approach can be used to compute standard errors and
confidence regions for the results of more complicated counterfactual
simulations, as long as the associated simulations are smooth
functions of the parameters. However, sometimes it might be more
natural to write simulations with outcomes that are not smooth in the
parameters. For example, the following code uses simulation to
calculate the change in the probability of $y$ from adding 0.1 to
$x$.

In [ ]:
function counterfactual_sim(β, x, S)
  function onesim()
    e = rand(Logistic(), size(x)[1])
    baseline= (x*β .+ e .> 0)
    counterfactual= ((x.+0.1)*β .+ e .> 0)
    mean(counterfactual.-baseline)
  end
  mean([onesim() for s in 1:S])
end
ForwardDiff.gradient(β->counterfactual_sim(β,x,10),βhat)

Here, the gradient is 0 because the simulation function is a
step-function. In this situation, an alternative to the delta method
is the simulation based approach of @krinsky1986. The procedure is
quite simple. Suppose 
$\sqrt{n}(\hat{\theta} - \theta_0) \indist N(0,\Omega)$,
and you want to an estimate of the distribution of $g(\theta)$. 
Repeatedly draw $\theta_s \sim N(\hat{\theta}, \Omega/n)$ and compute
$g(\theta_s)$. Use the distribution of $g(\theta_s)$ for
inference. For example, a 90% confidence interval for $g(\theta)$
would be the 5%-tile of $g(\theta_s)$ to the 95%-tile of
$g(\theta_s)$.

In [ ]:
Ω = avar.variance/n
Ω = (Ω+Ω')/2         # otherwise, it's not exactly symmetric due to
                     # floating point roundoff
function kr_confint(g, θ, Ω, simulations; coverage=0.9)
  θs = [g(rand(MultivariateNormal(θ,Ω))) for s in 1:simulations]
  quantile(θs, [(1.0-coverage)/2, coverage + (1.0-coverage)/2])
end

@show kr_confint(β->counterfactual_sim(β,x,10), βhat, Ω, 1000)

# a delta method based confidence interval for the same thing
function counterfactual_calc(β, x)
  baseline      = cdf.(Logistic(), x*β)
  counterfactual= cdf.(Logistic(), (x.+0.1)*β)
  return([mean(counterfactual.-baseline)])
end
v = delta_method(β->counterfactual_calc(β,x), βhat, Ω)
ghat = counterfactual_calc(βhat,x)
@show [ghat + sqrt(v)*quantile(Normal(),0.05), ghat +
       sqrt(v)*quantile(Normal(),0.95)]

# Alternative approaches to inference

## Identification robust inference

As discussed in section 9 of @newey1994, there are three classic types
of statistics for testing restrictions on parameters. Suppose you want
to test $H_0: a(\theta) = 0$. Let $\hat{\theta}$ denote the
unrestricted estimate, and let $\hat{\theta}^r$ denote the
estimate of $\theta$ subject to the restriction. Wald test-statistics
are based on $\hat{\theta}-\hat{\theta}^r$. Lagrange multiplier tests
look at the distribution of the estimated Lagrange
multiplier. Likelihood ratio (aka distance metric in @newey1994) tests
look at $Q_n(\hat{\theta}) - Q_n(\hat{\theta}^r)$. If we consider
testing $H_0: \theta = \vartheta$ for some fixed $\vartheta$, then the
usual approach based on the asymptotic normality of $\hat{\theta}$
discussed above is exactly the same as the Wald test of this
restriction. As discussed by @newey1994, under standard assumptions,
all three testing approaches are asymptotically equivalent. However,
the tests can and will differ in finite samples. More importantly, in
the face of identification problems, Wald tests tend to break down,
while Lagrange multiplier and likelihood ratio style tests can
continue to work. 

By identification robust, we mean an inference procedure that has
correct size regardless of whether identification is strong, weak,
or partial. In the asymptotic normality of extremum estimators theorem
above, non-strong identification will create problems for assumption
3, in particular the assumption that the Hessian is non-singular. For
this section, we will focus on GMM estimators. Identification problems
most often arrive and have been studied in the context of GMM. Also,
it is not difficult to transform other extremum estimators into GMM. 

For a GMM objective function of the form:
$$ [1/n \sum_i g_i(\theta)] W_n [1/n \sum g_i(\theta]$$, 
if we assume:
1. $1/\sqrt{n} \sum_i g_i(\theta_0) \indist N(0,\Sigma)$
2. $1/n \sum_i \nabla g_i(\theta) \inprob E[\nabla g(\theta)]$, 
   $W_n \inprob W$
3. $(D'WD)$ is nonsingular.
then the above theorem for asymptotic normality of extremum
estimators implies that 
$$
\sqrt{n}(\hat{\theta} - \theta_0) \indist N(0,\Omega)
$$
where 
$$
 \Omega= (D'WD)^{-1} (D' W \Sigma W D) (D'WD)^{-1}.
$$
If we additionally assume $W_n \inprob \Sigma^{-1}$, e.g. observations
are independent and $W_n =
\widehat{Var}(g_i(\theta))^{-1}$, then the asymptotic variance
simplifies to $(D' \Sigma D)^{-1}$. 

### Anderson-Rubin test

As already stated, the assumption that $(D'WD)$ is nonsingular is
problematic if we want to allow for identification problems. However,
if we assume only that 
1. $1/\sqrt{n} \sum_i g_i(\theta_0) \indist N(0,\Sigma)$
2. $W_n \inprob \Sigma^{-1}$
then 
$$
n [1/n \sum g_i(\theta_0)]' W_n^{-1} [1/n \sum g_i(\theta_0)]
  \indist \chi^2_m
$$
where $m$ is the number of moments (dimension of $g_i(\theta)$). This
is called the Anderson-Rubin test. Note that this result holds without
any explicit nonsingular assumption about a Hessian. Hence, there is
hope that this result would be true even with identification
problems. Indeed, it is. @stock2000 first proposed using this test
statistic for weakly identified GMM estimators. @stock2002 gives an
overview of this test and related tests with a focus on linear IV. 

Typical usage of the AR test is to invert the test to construct a
confidence region for $\theta$. For each $\theta \in \Theta$, 
let 
$$
AR(\theta) = n [1/n \sum g_i(\theta)]' \widehat{Var}(g_i(\theta))^{-1}
[1/n \sum g_i(\theta)] 
$$
and let $c_{\alpha}= \alpha$ quantile of $\chi^2_m$. Then a $\alpha$
confidence region for $\theta_0$ is
$$
\{ \theta \in \Theta: AR(\theta) \leq c_\alpha \}
$$

In [ ]:
function simulate_ivshare(n,β,π,ρ)
  z = randn(n, size(π)[1])
  endo = randn(n, length(β))
  x = z*π .+ endo 
  e = rand(Normal(0,sqrt((1-ρ^2))),n).+endo[:,1]*ρ
  y = cdf.(Logistic(), x*β .+ e)
  return((y=y,x=x,z=z))  
end
n = 500
k = 2
iv = 4
β0 = ones(k)
π0 = ones(iv,k)
ρ = 0.5
(y,x,z) = simulate_ivshare(n,β0,π0,ρ)

function gi_ivshare(β,y,x,z)
  e = quantile.(Logistic(),y) .- x*β
  e.*z
end

function gmmObj(θ,gi,W)
  g = gi(θ)
  m = mean(g,dims=1)
  (size(g)[1]*( m*W*m')[1]) # return scalar, not 1x1 array
end
function ar(θ,gi)
  gmmObj(θ,gi,inv(cov(gi(θ))))
end

optres = optimize(θ->gmmObj(θ,β->gi_ivshare(β,y,x,z),I),
                  zeros(k), NewtonTrustRegion(), autodiff =:forward)
@show β1 = optres.minimizer 
optres = optimize(θ->gmmObj(θ,β->gi_ivshare(β,y,x,z),inv(cov(gi_ivshare(β1,y,x,z)))),
                  zeros(k), NewtonTrustRegion(), autodiff =:forward)
@show βeff = optres.minimizer 
optres = optimize(θ->ar(θ,β->gi_ivshare(β,y,x,z)),
                  zeros(k), NewtonTrustRegion(), autodiff =:forward)
@show βcue = optres.minimizer

TO BE CONTINUED